In [2]:
import unittest
from pathlib import Path
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from tools.torch.trainers import TorchTrainer
from tools.torch.listeners import Listeners, TensorBoardLossReporter, TensorBoardModelReporter

In [3]:
test_dir = Path('test')
root = test_dir.joinpath('data')

In [4]:
device = torch.device('cpu')

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root=root,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=root,
    train=False,
    download=True,
    transform=ToTensor(),
)

In [7]:
model = NeuralNetwork()

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
checkpoint_dir = test_dir.joinpath('checkpoint')
log_dir = test_dir.joinpath('log')

In [11]:
listener = Listeners([
    TensorBoardLossReporter(log_dir),
    TensorBoardModelReporter(log_dir),
])

In [12]:
batch_size = 16
batch_multi = 1

trainer = TorchTrainer(
    name=f'mnist_{batch_size}_{batch_multi}',
    epochs=5,
    device=device,
    batch_size=batch_size,
    train_data=training_data,
    val_data=test_data,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    listener=listener,
    batch_multi=batch_multi,
    checkpoint_dir=checkpoint_dir,
)

In [13]:
trainer.start()

epoch: 5, step: 625, loss: 0.292: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:04<00:00, 131.61it/s]
